## Redes Neurais Artificiais 2025.1

- **Disciplina**: Redes Neurais Artificiais 2025.1  
- **Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
- **Github**: http://github.com/elloa  
        

Levando em conta a base de dados **_Forest Cover Type_**, esta segunda parte do Projeto Prático 2.2 diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Testando Redes Neurais sem os Atributos Categórios

1. Abra a base de dados em questão
2. Elimine todas as colunas relativas aos atributos categóricos
3. Armazene o atributo alvo em uma variável y e os atributos preditores em uma variável X
4. Efetue uma partição holdout 70/30 com o sklearn, distribuindo os exemplos de maneira aleatória
5. Efetue o escalonamento dos atributos

### Escalonando os atributos

O treinamento de uma rede neural artificial é mais eficiente quando os valores que lhes são fornecidos como entrada são pequenos, pois isto favorece a convergência. Isto é feito escalonando-se todos os atributos para o intervalo [0,1], mas precisa ser feito de maneira cautelosa, para que informações do conjunto de teste não sejam fornecidas no treinamento.

Há duas estratégias para tal escalonamento: normalização e padronização. Ambas possuem características particulares, vantagens e limitações, como é possível ver aqui: https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/


No nosso caso, vamos usar a padronização. Assim, com os atributos preditores do treinamento, isto é, X_train, deve-se subtrair a média e dividir pelo desvio padrão:

X_train_std = (X_train - np.mean(X_train))/np.std(X_train)

Em seguida, o mesmo deve ser feito com os atributos preditores do conjunto de testes, mas com padronização relativa ao conjunto de treinamento:

X_test_std = (X_test - np.mean(X_train))/np.std(X_train)

Se todo o conjunto X for utilizado na padronização, a rede neural receberá informações do conjunto de teste por meio da média e variância utilizada para preparar os dados de treinamento, o que não é desejável.

In [17]:
## Reservado para a importação de bibliotecas
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, ConfusionMatrixDisplay, confusion_matrix
import joblib
from joblib import Parallel, delayed
from joblib import dump
from itertools import product

In [ ]:
# 1 - Abrindo a base de dados

df = pd.read_csv('covtype.csv')
df.head(10)

In [19]:
# 2 - Removendo as colunas relacionadas aos atributos categóricos

atributos_categoricos = ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
                        'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 
                        'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 
                        'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 
                        'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 
                        'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 
                        'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 
                        'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 
                        'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 
                        'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 
                        'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']

df.drop(columns = atributos_categoricos, inplace = True)

In [ ]:
# 3 - Armazenando o atributo alvo em uma variável y 
# e os atributos preditores em uma variável X

X, y = df.drop(columns = 'Cover_Type'), df['Cover_Type']
X

In [22]:
# 4 e 5 - Criação de uma função para efetuar uma partição holdout 70/30
# e realizar a Padronização dos conjuntos de treino e teste:

def holdout_partition (X, y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

    X_train_std = (X_train - np.mean(X_train))/np.std(X_train)

    X_test_std = (X_test - np.mean(X_train))/np.std(X_train)

    return X_train, X_test, y_train, y_test, X_train_std, X_test_std

In [ ]:
X_train, X_test, y_train, y_test, X_train_std, X_test_std = holdout_partition(X, y)

### Continuando

5. Treine uma rede neural multilayer perceptron para este problema com uma única camada e dez neurônios  
    5.1 Utilize a função de ativação ReLU  
    5.2 Utilize o solver Adam    
    5.3 Imprima o passo a passo do treinamento    
    5.4 Utilize o número máximo de épocas igual a 300  
6. Com o modelo em questão, após o treinamento, apresente:  
    6.1 Matriz de confusão para o conjunto de teste  
    6.2 Acurácia  
    6.3 F-Score  
    6.4 Precisão  
    6.5 Revocação  
7. Repita o treinamento da mesma rede anterior sem imprimir o passo a passo (verbose False) por 100 vezes  
    7.1 Cada uma destas repetições deve ser feita com uma nova partição Holdout  
    7.2 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino  
8. Repita por 100 vezes o treinamento desta mesma rede, mas utilizando o otimizador SGD  
    8.1 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino  
9. Houve influência da escolha do otimizador no desempenho da rede?

In [24]:
# 5 - Criação da rede neural MultiLayer Perceptron (MLP):

mlp = MLPClassifier(
    hidden_layer_sizes = (10,),     # 1 camada com 10 neurônios
    activation = 'relu',            # função de ativação ReLU
    solver = 'adam',                # solver Adam
    max_iter = 300,                 # 300 épocas
    verbose = True,                 # passo a passo do treinamento
)

In [ ]:
# Treinamento da rede neural MLP:

mlp.fit(X_train_std, y_train)

y_pred = mlp.predict(X_test_std)

In [ ]:
# 6 - Matriz de Confusão

fig, ax = plt.subplots(figsize = (8, 6))

disp = ConfusionMatrixDisplay.from_predictions(y_test, y_pred, ax = ax, cmap = 'Greens')

ax.set_xlabel("Rótulos Preditos", fontsize = 10)
ax.set_ylabel("Rótulos Reais", fontsize = 10)

ax.tick_params(axis = 'both', labelsize = 8)

plt.show()

In [ ]:
# 6 - Acurácia, Precisão, Recall e F1-Score

metrics_data = {
    'Acurácia': [accuracy_score(y_test, y_pred)],
    'Precisão': [precision_score(y_test, y_pred, average = 'weighted', zero_division = 0)],
    'Recall': [recall_score(y_test, y_pred, average = 'weighted', zero_division = 0)],
    'F1-Score': [f1_score(y_test, y_pred, average = 'weighted', zero_division = 0)]
}

# Criar DataFrame e exibir em porcentagem
df_metricas = pd.DataFrame(metrics_data, index = ['MLP 1']).T * 100

# Formatar os valores para 2 casas decimais e adicionar '%'
df_metricas = df_metricas.applymap(lambda x: f"{x:.2f}%")

print(df_metricas)

In [28]:
# 7 - Repetindo o treinamento da rede neural MLP anterior 100 vezes, com verbose = False :

def treino_holdout_adam(X, y):
    X_train, X_test, y_train, y_test, X_train_std, y_test_std = holdout_partition(X, y)
    
    mlp = MLPClassifier(
    hidden_layer_sizes = (10,),     # 1 camada com 10 neurônios
    activation = 'relu',            # função de ativação ReLU
    solver = 'adam',                # solver Adam
    max_iter = 300,                 # 300 épocas
    verbose = False,                # sem passo a passo do treinamento
    )

    mlp.fit(X_train_std, y_train)
    y_pred = mlp.predict(X_train_std)
    
    acc = accuracy_score(y_train, y_pred)
    f1 = f1_score(y_train, y_pred, average = 'weighted', zero_division = 0)
    return acc, f1

# Executar em paralelo (usa todos os núcleos disponíveis)
resultados_adam = Parallel(n_jobs = -1)(delayed(treino_holdout_adam)(X, y) for _ in range(100))

# Separar métricas
accs_adam, f1s_adam = zip(*resultados_adam)

In [ ]:
print("Resultados - Adam (Treino)")
print(f"Acurácia - Média: {np.mean(accs_adam):.4f}, Desvio Padrão: {np.std(accs_adam):.4f}")
print(f"F1-Score - Média: {np.mean(f1s_adam):.4f}, Desvio Padrão: {np.std(f1s_adam):.4f}")

In [30]:
# 8 - Repetindo o treinamento com solver = "sgd":

def treino_holdout_sgd(X, y):
    X_train, X_test, y_train, y_test, X_train_std, y_test_std = holdout_partition(X, y)
    
    mlp = MLPClassifier(
        hidden_layer_sizes = (10,),     # 1 camada com 10 neurônios
        activation = 'relu',            # função de ativação ReLU
        solver = 'sgd',                # solver Adam
        max_iter = 300,                 # 300 épocas
        verbose = False,                # sem passo a passo do treinamento
    )
    
    mlp.fit(X_train_std, y_train)
    y_pred = mlp.predict(X_train_std)

    acc = accuracy_score(y_train, y_pred)
    f1 = f1_score(y_train, y_pred, average = 'weighted', zero_division = 0)

    return acc, f1

resultados_sgd = Parallel(n_jobs = -1)(delayed(treino_holdout_sgd)(X, y) for _ in range(100))

accs_sgd, f1s_sgd = zip(*resultados_sgd)

In [ ]:
print("Resultados - SGD (Treino)")
print(f"Acurácia - Média: {np.mean(accs_sgd):.4f}, Desvio Padrão: {np.std(accs_sgd):.4f}")
print(f"F1-Score - Média: {np.mean(f1s_sgd):.4f}, Desvio Padrão: {np.std(f1s_sgd):.4f}")

In [ ]:
if np.mean(accs_adam) > np.mean(accs_sgd):
    print("\nAdam obteve melhor desempenho médio que SGD.")
else:
    print("\nSGD obteve melhor desempenho médio que Adam.")

# Diferença clara?
print(f"Diferença média de acurácia: {np.mean(accs_adam) - np.mean(accs_sgd):.4f}")
print(f"Diferença média de F1-score: {np.mean(f1s_adam) - np.mean(f1s_sgd):.4f}")

In [ ]:
print("Maior valor:", np.max(X))

#### 9 - Houve influência da escolha do otimizador no desempenho da rede?

A escolha do otimizador influencia significativamente o desempenho de uma rede neural, tanto em termos de acurácia quanto na velocidade de convergência durante o treinamento. Otimizadores determinam como os pesos da rede são atualizados com base no gradiente do erro, afetando diretamente a eficiência do aprendizado. Por exemplo, o otimizador Adam (Adaptive Moment Estimation) combina as vantagens de métodos como RMSProp e Momentum, adaptando dinamicamente as taxas de aprendizado para cada parâmetro com base em estimativas de momentos de primeira e segunda ordem. Isso o torna mais eficaz em muitos cenários, especialmente quando há dados ruidosos ou funções de custo complexas, o que pode explicar por que ele superou o SGD (Stochastic Gradient Descent) em seu teste. Já o SGD é mais simples e pode ser eficaz, mas costuma exigir ajustes mais cuidadosos da taxa de aprendizado e mais épocas para alcançar bons resultados.

## Discussão

Nos passos anteriores, você avaliou o desempenho de uma única rede neural que contém os seguintes parâmetros: uma única camada oculta com 10 neurônios e função de ativação ReLU. O otimizador utilizado, quer seja SGD ou ADAM, trata-se do algoritmo para aproximar o gradiente do erro. Neste sentido, a escolha do otimizador é um hiperparâmetro, pois diz respeito a como a rede neural definida previamente atuará "em tempo de execução"  durante o processo de treinamento. Também são hiperparâmetros a quantidade de épocas, a taxa de aprendizado inicial, dentre outros.

Cabe alientar também que você efetuou o treinamento desta rede por 100 vezes e apresentou os resultados em termos de média +- desvio padrão. Lembre-se que em uma rede neural há a inicialização aleatória de pesos e, em consequência, o desempenho delas está sujeito à uma flutuação estocástica. A execução destas múltiplas vezes faz com que eliminemos algum viés introduzido por uma boa ou má "sorte" na escolha de pesos no caso de uma única execução.

Você também aprendeu uma estratégia para escalonar os atributos para uma melhor convergência da rede. Utilize-a em todos os treinamentos e testes propostos a seguir.

## Propondo Novas Arquiteturas

Variando  os parâmetros (uma ou duas camadas ocultas, com diferente números de neurônios em cada uma delas e a função de ativação) e o hiperparâmetros solver (Adam ou SGD) e o número de épocas (100,150 e 200), atenda ao que se pede:

1. Proponha 10 arquiteturas distintas de RNAs para o problema em questão, à sua escolha
2. Avalie cada uma das arquiteturas perante todos os hiperparâmetros apresentados por 100 vezes
3. Como resultado da avaliação, apresente:  
    3.1 Top-3 melhores redes no tocante à F-Score e Acurácia  
    3.2 Repetição em que houve o melhor desempenho de cada uma dessas redes: ilustre tp, tf, fp e fn  

In [172]:
architectures = [(10,), (16,4), (20,), (4,8), (20,20), (30,), (8,8), (16,16), (25,10), (12,6)]

functions = ['relu', 'logistic']

solvers = ['adam', 'sgd']

epochs = [100, 150, 200]

grid = list(product(architectures, functions, solvers, epochs))

In [173]:
def get_top3(overall_averages):
    top3 = sorted(overall_averages, key = lambda av: (av[1], av[0]), reverse = True)[:3]

    print("\nTop-3 modelos por Acurácia e F1-score (médias):\n")

    for i, (f1, acc, idx, _, _) in enumerate(top3):
        print(f"Top {i + 1}: Arquitetura {idx} => Acurácia = {acc:.4f} | F1-score =  {f1:.4f} | Modelo = {grid[idx]}")

    return top3

In [174]:
# Função que gera os modelos de MLPClassifier com base na arquitetura passada como parâmetro, retornando os valores de acurácia de f1-score

def treino_e_avaliacao(X, y, arquitetura):
    X_train, X_test, y_train, y_test, X_train_std, X_test_std = holdout_partition(X, y)
    
    mlp = MLPClassifier(
        hidden_layer_sizes = arquitetura[0],
        activation = arquitetura[1],
        solver = arquitetura[2],
        max_iter = arquitetura[3],
        verbose = False
    )

    mlp.fit(X_train_std, y_train)
    y_pred = mlp.predict(X_test_std)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average = 'weighted', zero_division = 0)
    cm = confusion_matrix(y_test, y_pred)

    return acc, f1, cm, y_test, y_pred, mlp

In [ ]:
# Diretório onde os modelos serão salvos
os.makedirs("modelos_melhores_top48", exist_ok = True)

# Resultados detalhados para análise posterior
media_geral_arquiteturas = []
resultados_detalhados = []


for idx, model in enumerate(grid):
    print(f"Executando modelo {idx + 1}/{len(grid)}: {model}")

    resultados = Parallel(n_jobs = -1)(delayed(treino_e_avaliacao)(X, y, model) for _ in range(100))

    acuracias = [r[0] for r in resultados]
    f1_scores = [r[1] for r in resultados]

    # Melhor execução com base em F1 (desempate por acurácia)
    melhor_resultado = max(resultados, key = lambda r: (r[1], r[0]))
    acc, f1, cm, y_test, y_pred, mlp_model = melhor_resultado

    # Salvar o melhor modelo no disco
    nome_arquivo = f"modelos_melhores_top48/modelo_{idx:02d}_acc{acc:.4f}_f1{f1:.4f}.joblib"
    dump(mlp_model, nome_arquivo)

    media_acc = np.mean(acuracias)
    media_f1 = np.mean(f1_scores)
    desvio_acc = np.std(acuracias)
    desvio_f1 = np.std(f1_scores)

    acc_from_100, f1_from_100, cm, ytest_from_100, ypred_from_100, mlp_from_100 = zip(*resultados)

    # Armazenar resultados
    media_geral_arquiteturas.append((media_f1, media_acc, idx, desvio_f1, desvio_acc))
    resultados_detalhados.append({
        'modelo': model,
        'acuracia': acc,
        'f1_score': f1,
        'cm': cm,
        'y_test': y_test,
        'y_pred': y_pred
    })

In [ ]:
# Mostra top-3 com matrizes
top3_resultados = get_top3(media_geral_arquiteturas)

In [ ]:
# Exibir matrizes de confusão para as 3 melhores
cmaps = ['Blues', 'Greens', 'Reds']

#    top3 = sorted(overall_averages, key = lambda av: (av[1], av[0]), reverse = True)[:3]

for i, (_, _, idx, _, _) in enumerate(top3_resultados):
    melhor = resultados_detalhados[idx]
    print(f"\nTop {i + 1} - Modelo: {melhor['modelo']}")
    print(f"Melhor iteração - Acurácia: {melhor['acuracia']:.4f}, F1-Score: {melhor['f1_score']:.4f}")
    disp = ConfusionMatrixDisplay.from_predictions(melhor['y_test'], melhor['y_pred'], cmap = cmaps[i])
    disp.ax_.set_title(f'Matriz de Confusão - Top {i + 1}')
    plt.show()

## Estimando o número de neurônios

Um dos problemas de pesquisa com redes neurais artificiais consiste na determinação do número de neurônios em sua arquitetura. Embora não seja possível definir a priori qual rede neural é adequada para um problema, pois isto só é possível mediante uma busca exaustiva, há regras na literatura que sugerem o número de neurônios escondidos, tal como a regra da Pirâmide Geométrica, dada a seguir:

$$N_h = \alpha \cdot \sqrt{N_i \cdot N_o},$$

em que $N_h$ é o número de neurônios ocultos (a serem distribuídos em uma ou duas camadas ocultas), $N_i$ é o número de neurônios na camada de entrada e $N_o$ é o número de neurônios na camada de saída. 

1. Consulte a documentação da classe MLPClassifier (disponível em https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) e obtenha os valores de $N_i$ e $N_o$.
2. Teste os valores de $\alpha$ como sendo iguais a $0.5$, $2$ e $3$.
3. Proponha pelo menos 30 redes neurais segundo a regra da pirâmide geométrica e teste-as nos mesmos termos estabelecidos anterioremente  (solver, épocas, etc.)  
    3.1 Apresente as top-3 melhores redes no tocante à F-Score e Acurácia  

O valor de $N_i$ deve ser igual a quantidade de características de entrada do treinamento e o $N_0$ deve ser igual a quantidade de classes distintas, tedo em vista que este é um problema de classificação multiclasse. Assim:


In [165]:
modelos = [
    ((10, 10), 'relu', 'adam', 100), 
    ((16, 4), 'tanh', 'sgd', 100), 
    ((10, 10), 'identity', 'adam', 150),
    ((20,), 'relu', 'adam', 200), 
    ((16, 4), 'tanh', 'adam', 100), 
    ((8, 8), 'logistic', 'adam', 150),
    ((30,), 'identity', 'adam', 150), 
    ((20, 20), 'relu', 'sgd', 200), 
    ((25, 10), 'identity', 'adam', 200),
    ((12, 6), 'identity', 'sgd', 150)
]

In [166]:
n_i = df.shape[1]
n_i

11

In [167]:
n_0 = y.nunique()
n_0

7

In [168]:
alphas = [0.5, 2, 3]

In [169]:
def criar_novas_arqs_com_nh(alphas, models):
    novas_arqs = []
    
    for alpha in alphas:
        n_h = int(alpha * np.sqrt(n_i * n_0))  # Cálculo do número de neurônios na camada escondida

        for arq in models:
            hidden_layer = (n_h,)  # Garantimos que seja tupla
            activation = arq[1]
            solver = arq[2]
            max_iter = arq[3]

            nova_arquitetura = (hidden_layer, activation, solver, alpha, max_iter)
            novas_arqs.append(nova_arquitetura)
    
    return novas_arqs

In [170]:
def treino_e_avaliacao_completo(X, y, arquitetura):
    X_train, X_test, y_train, y_test, X_train_std, X_test_std = holdout_partition(X, y)

    mlp = MLPClassifier(
        hidden_layer_sizes=arquitetura[0],
        activation=arquitetura[1],
        solver=arquitetura[2],
        alpha=arquitetura[3],  # inclui alpha aqui
        max_iter=arquitetura[4],
        verbose=False
    )
    
    mlp.fit(X_train_std, y_train)
    y_pred = mlp.predict(X_test_std)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average = 'weighted', zero_division = 0)
    cm = confusion_matrix(y_test, y_pred)

    return acc, f1, cm, y_test, y_pred, mlp

In [171]:
# Diretório onde os modelos com NH serão salvos
os.makedirs("modelos_nh", exist_ok = True)

media_geral_nh = []  # (f1, acc, idx, std_f1, std_acc)
resultados_detalhados_nh = []  # salva os melhores modelos com y_test, y_pred e matriz

novas_arqs = criar_novas_arqs_com_nh(alphas, modelos)

In [ ]:
for j in range(len(novas_arqs)):  
    print(f"\nExecutando arquitetura {j + 1}/{len(novas_arqs)}...")

    # Paralelização das 100 iterações
    resultados = Parallel(n_jobs = -1)(
        delayed(treino_e_avaliacao_completo)(X, y, novas_arqs[j]) for _ in range(100)
    )

    # Desempacotar os resultados
    acuracias = [r[0] for r in resultados]
    f1_scores = [r[1] for r in resultados]

    # Encontrar o melhor resultado (maior F1, desempate por acurácia)
    melhor_resultado = max(resultados, key=lambda r: (r[1], r[0]))

    media_acc = np.mean(acuracias)
    media_f1 = np.mean(f1_scores)
    std_acc = np.std(acuracias)
    std_f1 = np.std(f1_scores)

    # Armazena a média geral da arquitetura
    media_geral_nh.append((media_f1, media_acc, j, std_f1, std_acc))

    # Armazena o melhor caso completo para exibição posterior
    resultados_detalhados_nh.append({
        'arquitetura': novas_arqs[j],
        'acuracia': melhor_resultado[0],
        'f1_score': melhor_resultado[1],
        'cm': melhor_resultado[2],
        'y_test': melhor_resultado[3],
        'y_pred': melhor_resultado[4]
    })

    # Salvar o melhor modelo
    nome_arquivo = f"modelos_nh/melhor_modelo_nh_{j:02d}.joblib"
    dump(melhor_resultado[5], nome_arquivo)

In [ ]:
top3_nh = get_top3(media_geral_nh)

In [ ]:
cmaps = ['Blues', 'Greens', 'Reds']

for i, (_, _, idx, _, _) in enumerate(top3_nh):
    melhor = resultados_detalhados_nh[idx]
    print(f"\nTop {i + 1} - Modelo: {melhor['arquitetura']}")
    print(f"Melhor iteração - Acurácia: {melhor['acuracia']:.4f}, F1-Score: {melhor['f1_score']:.4f}")
    disp = ConfusionMatrixDisplay.from_predictions(melhor['y_test'], melhor['y_pred'], cmap = cmaps[i])
    disp.ax_.set_title(f'Matriz de Confusão - Top {i}')
    plt.show()

## Testando as Redes Neurais com Atributos Categóricos

1. Considere as 6 redes neurais obtidas nos dois top-3 anteriores (arquiteturas próprias e regra da pirâmide geométrica)
2. Com todos os atributos preditores da base de dados original, incluindo os categóricos, treine e teste estas mesmas redes por 100 repetições  
    2.1 Considere o melhor otimizador para cada uma delas  
    2.2 Faça uso de 200 épocas para treinamento  
    2.2 Apresente os resultados de acurácia e F-Score em termos da média +- dp para cada arquitetura
3. Apresente o gráfico boxplot para o F-Score das 6 arquiteturas perante as 100 repetições

In [111]:
top6 = top3_resultados + top3_nh

In [103]:
# Obter os dados sem os valores excluidos anteriormente
new_df = pd.read_csv('covtype.csv')
new_X, new_y = new_df.drop(columns = 'Cover_Type'), df['Cover_Type']

In [113]:
# Diretório para salvar os melhores modelos por arquitetura
os.makedirs("modelos_top6", exist_ok = True)

In [ ]:
# Nova funcao de teste com a variável max_iter predefinida para todos como 200

def training_geometric_pyamid(X, y, model):
    X = np.nan_to_num(X, nan = 0.0, posinf=np.finfo(np.float64).max, neginf=np.finfo(np.float64).min)
    X_train, X_test, y_train, y_test, X_train_std, X_test_std = holdout_partition(X, y)
    
    mlp = MLPClassifier(
        hidden_layer_sizes = model[0],
        activation = model[1],
        solver = model[2],
        max_iter = 200,
        verbose = False
    )

    mlp.fit(X_train_std, y_train)
    y_pred = mlp.predict(X_test_std)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    return acc, f1, mlp

In [ ]:
# === Execução das 100 repetições para as top 6 arquiteturas ===
resultados_top6 = []           # Guarda média e dp de acc/f1
f1_scores_por_arq = []         # Lista de listas para boxplot
melhores_modelos_info = []     # Guarda info dos melhores modelos

for i in range(6):
    if i < 3:
        print(f"\nExecutando arquitetura {i + 1}/6: {resultados_detalhados[idx]['modelo']}")
        resultados = Parallel(n_jobs=-1)(
            delayed(training_geometric_pyamid)(new_X, new_y, resultados_detalhados[idx]['modelo']) for _ in range(100)
        )
    else:
        print(f"\nExecutando arquitetura {i + 1}/6: {resultados_detalhados_nh[idx]['arquitetura']}")
        resultados = Parallel(n_jobs=-1)(
            delayed(training_geometric_pyamid)(new_X, new_y, resultados_detalhados_nh[idx]['arquitetura']) for _ in range(100)
        )

    acuracias = [r[0] for r in resultados]
    f1scores = [r[1] for r in resultados]

    melhor_resultado = max(resultados, key=lambda r: (r[1], r[0]))  # (acc, f1, modelo)
    acc_melhor, f1_melhor, melhor_modelo = melhor_resultado

    acc_melhor = melhor_resultado[0]
    f1_melhor = melhor_resultado[1]

    # Salva o melhor modelo da arquitetura
    nome_arquivo = f"modelos_top6/modelo_top{i+1}_acc{acc_melhor:.4f}_f1{f1_melhor:.4f}.joblib"
    joblib.dump(melhor_modelo, nome_arquivo)

    # Armazena resultados agregados
    resultados_top6.append((
        np.mean(acuracias), np.std(acuracias),
        np.mean(f1scores), np.std(f1scores)
    ))
    f1_scores_por_arq.append(f1scores)

    print(f"Arquitetura {i+1}: Acurácia = {np.mean(acuracias):.4f} ± {np.std(acuracias):.4f}, "
          f"F1-Score = {np.mean(f1scores):.4f} ± {np.std(f1scores):.4f}")

In [ ]:
# === 3. Boxplot do F1-Score ===

df_f1 = pd.DataFrame({
    f'Arquitetura {i + 1}': f1_scores_por_arq[i] for i in range(6)
})
df_f1_melted = df_f1.melt(var_name = 'Arquitetura', value_name = 'F1-Score')

In [ ]:
# Plot
plt.figure(figsize = (10, 6))
sns.boxplot(data = df_f1_melted, x = 'Arquitetura', y = 'F1-Score')
plt.title("Distribuição do F1-Score (100 execuções por arquitetura)")
plt.xticks(rotation = 45)
plt.grid(True)
plt.tight_layout()
plt.show()

## Considerações Parciais

1. É possível identificar uma rede com desempenho superior às demais?
2. Qual estratégia mostrou-se mais producente para a obtenção de boas arquiteturas (Estratégia Própria ou Pirâmide Geométrica)? Por quê?
3. Considerar os atributos categóricos trouxe melhorias? Justifique.
4. Um número maior de épocas trouxe melhorias?
5. Qual a maior dificuldade de resolução do problema proposto perante as RNAs?